# Market クラス解説

Marketクラスは、取引所で扱われる通貨ペアーの１組合わせをモデリングしたクラスです。
現在はBinanceの USDBUSDのSPOT取引しか扱えませんが、比較的容易に拡張できるようにつくっていますので順次拡張していきます。

## Marketクラスメソッド

### マーケットオブジェクトの生成(`open`)

`open`メソッドでMaketオブジェクトを開きます。内部的にキャッシュされているので２回目からはキャッシュを返します。そのため複数回呼び出してOKです。

今後SPOT以外にも対応した場合、パラメータが変更になる可能性があります。
```
binance = Market.open(
    'BN',               # Binanceは 'BN'
    'BTCBUSD'
)
```

### データのダウンロード(`download`)

指定した日付あらあとのデータをダウンロードしてDBへ保存します。
BTCBUSDしか対応していないため効果はありませんが、`open`で開いたマーケットオブジェクト全てに対してダウンロード処理をします。

```
Market.download(
    5   #ダウンロード日数（N日前を指定)
    )
```


## Marketオブジェクトメソッド

Marketインスタンスが`binance`オブジェクトに保存されている前提で説明します。

###　情報表示　

####　`_repr_html_()`

jupyuter note上でオブジェクトを入力してリターンすると内部的に`_repr_html_()`がよばれて内容が分かりやすく表示されます。
```
binance
```

#### DBのパス表示 `file_path`

DBは巨大になりますので、不要になったら`file_path`プロパティで表示される場所を確認して手動で消してください。

```
binance.file_name
```

### OHLCV取得（pd.DataFrame形式)

時刻はμsで指定する。開始時刻に0を指定するとDBの最初から、終了に0を指定するとDBの最後まで（現在時刻に一番ちかいときまで）を指定したことになる。

```
binance.ohlcv(
    start_timestamp_us,     #開始時刻（μs）
    end_timestamp_us,       #終了時刻（μs)
    60                      #足の幅（秒）
    )
```

pd.DataFrame 形式でOHLCVデータが取得できます。この機能だけつかっても便利に使えます。
なお、Tickデータから任意の足を再生成しているので初回キャッシュされていないときはかなり時間がかかります。




## 以下うごかしながら説明

In [2]:
# 必要ライブラリのimport
import rbot
from rbot import Market
from rbot import DAYS_BEFORE

In [6]:
# Binance('BN')の'BTCBUSD'ペアーの取引マーケットを生成
binance = Market.open('BN', 'BTCBUSD')

# 上記でオープンしたマーケットに対し5日より新しいログをダウンロード
Market.download(5)

Market.download(5, True)       #ディフォルトではキャッシュされているので、再ダウンロードしたい場合は強制フラグをTrueにする。

Start download from 2022-12-13T08:19:23.323697
download 0[rec]
Start download from 2022-12-13T08:19:23.323978
Downloaded rec = 3577204 
Downloading... 2022-12-13T08:07 / rec=   1000500Downloaded rec = 3067544 
Downloading... 2022-12-13T13:29 / rec=   2001000Downloaded rec = 2505203 
Downloading... 2022-12-13T17:45 / rec=   3001500Downloaded rec = 2830841 
Downloading... 2022-12-14T14:40 / rec=   5003917Downloaded rec = 1738925 
Downloading... 2022-12-17T12:40 / rec=  13009306download 13719717[rec]


In [4]:
# Marketオブジェクトが管理するDBに保存されているデータ期間を表示する。
binance

start,end
1670889600100000,1671407999900000
"""2022-12-13T00:00:00.100000""","""2022-12-18T23:59:59.900000"""
days=,5


In [7]:
# データベースのDisk上の場所を表示する。

binance.file_name

'/Users/takeoyasukawa/Library/Application Support/net.takibi.rbot/DB/BN/SPOT-BTCBUSD.db'

## OHLCVの取得

In [9]:
# 初回は遅いです。２回目はキャッシュされているので比較的高速です
binance.ohlcv(
    DAYS_BEFORE(5),     # 5日前から
    0,                  # 最新まで
    60*60
)

,open,high,low,close,volume
timestamp,,,,,
2022-12-14 08:00:00+00:00,17803.03,17878.00,17797.59,17840.61,3177.04731
2022-12-14 09:00:00+00:00,17840.28,17847.37,17785.11,17814.51,3196.71892
2022-12-14 10:00:00+00:00,17815.24,17837.75,17792.93,17828.15,3083.00192
2022-12-14 11:00:00+00:00,17828.15,17859.00,17807.00,17825.88,3778.80179
2022-12-14 12:00:00+00:00,17824.90,17930.00,17815.21,17914.61,6847.23861
...,...,...,...,...,...
2022-12-18 19:00:00+00:00,16748.27,16771.86,16725.01,16753.31,2477.76072
2022-12-18 20:00:00+00:00,16753.20,16774.87,16749.09,16759.96,2279.32264
2022-12-18 21:00:00+00:00,16759.68,16785.00,16738.11,16749.24,2285.04540


このOHCLVだけを利用して独自の分析を行うにも便利だと思います。

ご活用ください。